In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
df=pd.read_csv('onlinefraud.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [54]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt 
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score
from sklearn.preprocessing import LabelEncoder


In [55]:
df.shape


(6362620, 11)

In [56]:
df['balanceDiffOrig'] = df['oldbalanceOrg'] - df['newbalanceOrig'] 
df['balanceDiffDest'] = df['newbalanceDest'] - df['oldbalanceDest']

In [57]:
encoder = LabelEncoder()
df['type'] = encoder.fit_transform(df['type'])
df['type'].value_counts()

type
1    2237500
3    2151495
0    1399284
4     532909
2      41432
Name: count, dtype: int64

In [58]:
features = ['step','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','balanceDiffOrig','balanceDiffDest','type','isFlaggedFraud']
target = 'isFraud'

In [60]:
x = df[features]
y = df[target]

In [61]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [62]:
model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1) 
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [63]:
y_pred = model.predict(x_test) 
y_pred_proba = model.predict_proba(x_test)[:, 1] 

In [64]:
print(classification_report(y_test, y_pred)) 
print("ROC-AUC Score:", roc_auc_score(y_test,y_pred))
print("accuracy:",accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.98      0.77      0.86      2435

    accuracy                           1.00   1908786
   macro avg       0.99      0.88      0.93   1908786
weighted avg       1.00      1.00      1.00   1908786

ROC-AUC Score: 0.884381923297635
accuracy: 0.9996804251498073


In [78]:
import joblib
from sklearn.datasets import load_iris

In [80]:
joblib.dump(model, "fraud_model.pkl")
print("Model saved as fraud_model.pkl")

Model saved as fraud_model.pkl
